In [176]:
import xlsxwriter
import pandas
import re

In [177]:
csv_file_name = 'results_total.csv'

def read_csv_results(csv_file_name):
    data = pandas.read_csv(csv_file_name, sep = ';', encoding='latin-1')
    data_dictionary = data.to_dict()
    return data, data_dictionary

data, data_dictionary = read_csv_results(csv_file_name)

keys = list(data_dictionary.keys())

key_status = keys[0]
key_task_type = keys[1]
key_topology_name = keys[2]
key_dataset = keys[3]
key_train_framework = keys[4]
key_inference_framework = keys[5]
key_blob_size = keys[6]
key_precision = keys[7]
key_batch_size = keys[8]
key_execution_mode = keys[9]
key_parameters = keys[10]
key_infrastructure = keys[11]
key_avgtime = keys[12]
key_latency = keys[13]
key_fps = keys[14]

print('Available keys: \n \
\t{0}, {1}, {2}, {3}, {4}, \n \
\t{5}, {6}, {7}, {8}, {9}, \n \
\t{10}, {11}, {12}, {13}, {14}'.format(key_status, key_task_type, key_topology_name, \
                                 key_dataset, key_train_framework, \
                                 key_inference_framework, key_blob_size, \
                                 key_precision, key_batch_size, \
                                 key_execution_mode, key_parameters, key_infrastructure,\
                                 key_avgtime, key_latency, key_fps))

Available keys: 
 	Status, Task type, Topology name, Dataset, Framework, 
 	Inference Framework, Input blob sizes, Precision, Batch size, Mode, 
 	Parameters, Infrastructure, Average time of single pass (s), Latency, FPS


In [178]:
excel_file_name = 'test.xlsx'
excel_sheet_name = 'Performance'

book = xlsxwriter.Workbook(excel_file_name)
sheet = book.add_worksheet(excel_sheet_name)

cell_format = book.add_format({'align': 'center', 'valign': 'vcenter', \
                               'border': 1, 'bold': True, 'text_wrap': True})
sheet.merge_range('A1:A5', key_task_type, cell_format)
sheet.merge_range('B1:B5', key_topology_name, cell_format)
sheet.merge_range('C1:C5', key_train_framework, cell_format)
sheet.merge_range('D1:D5', key_blob_size, cell_format)
sheet.merge_range('E1:E5', key_batch_size, cell_format)

In [179]:
# get unique infrastructure
infrastructure = list(set(list(data_dictionary[key_infrastructure].values())))
inference_frameworks = []

for machine in infrastructure:
    machine_inference_frameworks = []
    for key, value in data_dictionary[key_inference_framework].items():
        if data_dictionary[key_infrastructure][key] == machine and \
           (value in machine_inference_frameworks) == False:
            machine_inference_frameworks.append(value)
    inference_frameworks.append(machine_inference_frameworks)

devices = []
for idx in range(len(infrastructure)):
    machine = infrastructure[idx]
    machine_inference_frameworks = inference_frameworks[idx]
    #print(machine)
    #print(machine_inference_frameworks)
    machine_framework_devices = []
    for inference_framework in machine_inference_frameworks:
        #print(inference_framework)
        framework_devices = []
        for key, value in data_dictionary[key_parameters].items():
            pattern = re.compile(r'[.]*Device:[ ]*(?P<device_name>[\W\w]+)[,]+[.]*')
            matcher = re.match(pattern, value)
            device_name = matcher.group('device_name')
            if data_dictionary[key_infrastructure][key] == machine and \
               data_dictionary[key_inference_framework][key] == inference_framework and \
               (device_name in framework_devices) == False:
                framework_devices.append(device_name)
        machine_framework_devices.append(framework_devices)
    devices.append(machine_framework_devices)

print(infrastructure)
print(inference_frameworks)
print(devices)

['CPU: Intel(R) Core(TM) i7-8700 CPU @ 3.20GHz, CPU family: x86_64, GPU: Intel(R) Gen9 HD Graphics (iGPU), RAM size: 65727428 kB, OS family: Linux, OS version: Linux-5.4.0-52-generic-x86_64-with-Ubuntu-18.04-bionic, Python version: 3.6.9', 'CPU: Intel(R) Core(TM) i3-8100 CPU @ 3.60GHz, CPU family: x86_64, GPU: Intel(R) Gen9 HD Graphics (iGPU), RAM size: 32757700 kB, OS family: Linux, OS version: Linux-5.4.0-73-generic-x86_64-with-Ubuntu-18.04-bionic, Python version: 3.6.9', 'CPU: Intel(R) Xeon(R) Gold 6138 CPU @ 2.00GHz CPU family: x86_64, GPU: None, RAM size: 526980036 kB, OS family: Linux OS version: Linux-5.8.0-53-generic-x86_64-with-Ubuntu-18.04-bionic, Python version: 3.6.9']
[['OpenVINO DLDT'], ['OpenVINO DLDT'], ['OpenVINO DLDT']]
[[['CPU', 'GPU', 'MYRIAD']], [['CPU', 'GPU', 'MYRIAD']], [['CPU']]]


In [180]:
precisions = []
for idx in range(len(infrastructure)):
    machine = infrastructure[idx]
    machine_inference_frameworks = inference_frameworks[idx]
    machine_precisions = []
    #print(machine)
    #print(machine_inference_frameworks)
    for idx2 in range(len(machine_inference_frameworks)):
        inference_framework = machine_inference_frameworks[idx2]
        framework_devices = devices[idx][idx2]
        framework_precisions = []
        #print(inference_framework)
        #print(framework_devices)
        for device in framework_devices:
            #print(device)
            device_precisions = []
            for key, value in data_dictionary[key_parameters].items():
                pattern = re.compile(r'[.]*Device:[ ]*(?P<device_name>[\W\w]+)[,]+[.]*')
                matcher = re.match(pattern, value)
                device_name = matcher.group('device_name')
                #print(data_dictionary['Infrastructure'][key], data_dictionary['Inference Framework'][key], device_name, data_dictionary['Precision'][key])
                if data_dictionary[key_infrastructure][key] == machine and \
                   data_dictionary[key_inference_framework][key] == inference_framework and \
                   device_name == device and \
                   (data_dictionary[key_precision][key] in device_precisions) == False:
                    device_precisions.append(data_dictionary[key_precision][key])
            #print(device_precisions)
            framework_precisions.append(device_precisions)
        machine_precisions.append(framework_precisions)
    precisions.append(machine_precisions)           

print(precisions)

[[[['FP32'], ['FP32', 'FP16'], ['FP16']]], [[['FP32'], ['FP32', 'FP16'], ['FP16']]], [[['FP32']]]]


In [181]:
execution_modes = []
for idx in range(len(infrastructure)):
    machine = infrastructure[idx]
    machine_inference_frameworks = inference_frameworks[idx]
    #print(machine)
    #print(machine_inference_frameworks)
    machine_modes = []
    for idx2 in range(len(machine_inference_frameworks)):
        inference_framework = machine_inference_frameworks[idx2]
        framework_devices = devices[idx][idx2]
        #print(inference_framework)
        #print(framework_devices)
        framework_modes = []
        for idx3 in range(len(framework_devices)):
            device = framework_devices[idx3]
            device_precisions = precisions[idx][idx2][idx3]
            #print(device)
            #print(device_precisions)
            framework_device_modes = []
            for precision in device_precisions:
                #print(precision)
                device_precision_modes = []
                for key, value in data_dictionary[key_parameters].items():
                    pattern = re.compile(r'[.]*Device:[ ]*(?P<device_name>[\W\w]+)[,]+[.]*')
                    matcher = re.match(pattern, value)
                    device_name = matcher.group('device_name')
                    if data_dictionary[key_infrastructure][key] == machine and \
                       data_dictionary[key_inference_framework][key] == inference_framework and \
                       device_name == device and \
                       data_dictionary[key_precision][key] == precision and \
                       (data_dictionary[key_execution_mode][key] in device_precision_modes) == False:
                        device_precision_modes.append(data_dictionary[key_execution_mode][key])
                #print(device_precision_modes)
                framework_device_modes.append(device_precision_modes)
            framework_modes.append(framework_device_modes)
        machine_modes.append(framework_modes)
    execution_modes.append(machine_modes)

print(execution_modes)

[[[[['Sync', 'Async']], [['Sync', 'Async'], ['Sync', 'Async']], [['Sync', 'Async']]]], [[[['Sync', 'Async']], [['Sync', 'Async'], ['Sync', 'Async']], [['Sync', 'Async']]]], [[[['Sync', 'Async']]]]]


In [182]:
rel_row_idx = 4
rel_col_idx = 5

col_indeces = []
for idx in range(len(infrastructure)):
    row_idx = rel_row_idx
    col_idx = rel_col_idx
    num_cols = 0
    machine = infrastructure[idx]
    # print(machine)
    machine_inference_frameworks = inference_frameworks[idx]
    col_idx3 = col_idx
    col_indeces2 = []
    for idx2 in range(len(machine_inference_frameworks)):
        machine_framework = machine_inference_frameworks[idx2]
        # print(machine_framework)
        machine_framework_devices = devices[idx][idx2]
        col_idx2 = col_idx
        num_cols2 = 0
        col_indeces3 = []
        for idx3 in range(len(machine_framework_devices)):
            machine_framework_device = machine_framework_devices[idx3]
            # print(machine_framework_device)
            framework_device_precisions = precisions[idx][idx2][idx3]
            col_idx1 = col_idx
            num_cols1 = 0
            col_indeces4 = []
            for idx4 in range(len(framework_device_precisions)):
                framework_device_precision = framework_device_precisions[idx4]
                # print(framework_device_precision)
                framework_device_precision_modes = execution_modes[idx][idx2][idx3][idx4]
                col_indeces5 = []
                for idx5 in range(len(framework_device_precision_modes)):
                    framework_device_precision_mode = framework_device_precision_modes[idx5]
                    # print(framework_device_precision_mode)
                    sheet.write(row_idx, col_idx + idx5, framework_device_precision_mode, cell_format)
                    col_indeces5.append(col_idx + idx5)
                sheet.merge_range(row_idx - 1, col_idx1, \
                                  row_idx - 1, col_idx1 + len(framework_device_precision_modes) - 1, \
                                  framework_device_precision, cell_format)
                col_idx += len(framework_device_precision_modes)
                col_idx1 += len(framework_device_precision_modes)
                num_cols1 += len(framework_device_precision_modes)
                num_cols2 += len(framework_device_precision_modes)
                num_cols += len(framework_device_precision_modes)
                col_indeces4.append(col_indeces5)
            sheet.merge_range(row_idx - 2, col_idx2, \
                              row_idx - 2, col_idx2 + num_cols1 - 1, \
                              machine_framework_device, cell_format)
            col_idx2 += num_cols1
            col_indeces3.append(col_indeces4)
        sheet.merge_range(row_idx - 3, col_idx3, \
                          row_idx - 3, col_idx3 + num_cols2 - 1, \
                          machine_framework, cell_format)
        col_idx3 += num_cols2
        col_indeces2.append(col_indeces3)
    sheet.merge_range(row_idx - 4, rel_col_idx, \
                      row_idx - 4, rel_col_idx + num_cols - 1, \
                      machine, cell_format)
    rel_col_idx += num_cols
    col_indeces.append(col_indeces2)

print(col_indeces)

[[[[[5, 6]], [[7, 8], [9, 10]], [[11, 12]]]], [[[[13, 14]], [[15, 16], [17, 18]], [[19, 20]]]], [[[[21, 22]]]]]


In [183]:
def remove_unused_metrics(data):
    # remove unused keys from DataFrame
    for key, value in data.copy().items():
        if key == key_avgtime or key == key_latency:
            del data[key]
    return data

def remove_failed_experiments(data):
    # transpose 2d dictionary
    experiments = data.to_dict('index')
    print('Full items: {0}'.format(len(experiments)))

    # remove failed experiments
    for key, value in experiments.copy().items():
        if value[key_status] == 'Failed':
            del experiments[key]

    print('Actual items: {0}'.format(len(experiments)))
    return experiments

def remove_unused_keys(data):
    data = remove_unused_metrics(data)  
    experiments = remove_failed_experiments(data)
    return experiments

experiments = remove_unused_keys(data)

Full items: 3528
Actual items: 3486


In [185]:
row_idx = 5

def find_idx(element, available_elements, exception_str):
    try:
        return available_elements.index(element)
    except:
        raise exception_str

def find_infrastructure_idx(infrastructure_name, available_infrastructure):
    return find_idx(infrastructure_name, available_infrastructure, \
                    'Unknown infrastructure')

def find_inference_framework_idx(framework_name, available_inference_frameworks):
    return find_idx(framework_name, available_inference_frameworks, \
                    'Unknown inference framework')

def find_device_idx(device_name, available_devices):
    return find_idx(device_name, available_devices, 'Unknown device name')

def find_precision_idx(precision, available_precisions):
    return find_idx(precision, available_precisions, 'Unknown precision')

def find_execution_mode_idx(execution_mode, available_execution_modes):
    return find_idx(execution_mode, available_execution_modes, \
                    'Unknown execution mode')

def find_column_idx(value, infrastructure, inference_frameworks, devices, \
                    precisions, execution_modes, col_indeces):
    idx1 = find_infrastructure_idx(value[key_infrastructure], infrastructure)    
    idx2 = find_inference_framework_idx(value[key_inference_framework], inference_frameworks[idx1])
    pattern = re.compile(r'[.]*Device:[ ]*(?P<device_name>[\W\w]+)[,]+[.]*')
    matcher = re.match(pattern, value[key_parameters])
    device_name = matcher.group('device_name')
    idx3 = find_device_idx(device_name, devices[idx1][idx2])
    idx4 = find_precision_idx(value[key_precision], precisions[idx1][idx2][idx3])
    idx5 = find_execution_mode_idx(value[key_execution_mode], execution_modes[idx1][idx2][idx3][idx4])
    return col_indeces[idx1][idx2][idx3][idx4][idx5]


for key, value in experiments.items():
    sheet.write(row_idx, 0, value[key_task_type])
    sheet.write(row_idx, 1, value[key_topology_name])
    sheet.write(row_idx, 2, value[key_train_framework])
    sheet.write(row_idx, 3, value[key_blob_size])
    sheet.write(row_idx, 4, value[key_batch_size])
    try:
        col_idx = find_column_idx(value, infrastructure, inference_frameworks, devices, \
                                  precisions, execution_modes, col_indeces)
        sheet.write(row_idx, col_idx, float(value[key_fps]))
    except Exception as ex:
        print('ERROR : {0}'.format(str(ex)))
    row_idx += 1

In [186]:
book.close()